In [7]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy

# load MATLAB files
from scipy.io import loadmat

pd.set_option('display.max_rows', 10)

%matplotlib inline

import seaborn as sns
sns.set_context('notebook')
sns.set_style('darkgrid')

## Load MATLAB datafiles

In [23]:
matfile = 'ex4/ex4data1.mat'
data1 = scipy.io.loadmat( matfile )
data1

{'X': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ..., 
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [25]:
X, y = data1['X'], data1['y']
#Insert a column of 1's to X for initial bias
X = np.insert(X,0,1,axis=1)
print ("'y' shape: %s. Unique elements in y: %s" %(data1['y'].shape,np.unique(data1['y'])))
print ("'X' shape: %s. X[0] shape: %s"%(X.shape,X[0].shape))

'y' shape: (5000, 1). Unique elements in y: [ 1  2  3  4  5  6  7  8  9 10]
'X' shape: (5000, 401). X[0] shape: (401,)


In [26]:
weights = loadmat('ex4/ex4weights.mat')
weights

{'Theta1': array([[ -2.25623899e-02,  -1.05624163e-08,   2.19414684e-09, ...,
          -1.30529929e-05,  -5.04175101e-06,   2.80464449e-09],
        [ -9.83811294e-02,   7.66168682e-09,  -9.75873689e-09, ...,
          -5.60134007e-05,   2.00940969e-07,   3.54422854e-09],
        [  1.16156052e-01,  -8.77654466e-09,   8.16037764e-09, ...,
          -1.20951657e-04,  -2.33669661e-06,  -7.50668099e-09],
        ..., 
        [ -1.83220638e-01,  -8.89272060e-09,  -9.81968100e-09, ...,
           2.35311186e-05,  -3.25484493e-06,   9.02499060e-09],
        [ -7.02096331e-01,   3.05178374e-10,   2.56061008e-09, ...,
          -8.61759744e-04,   9.43449909e-05,   3.83761998e-09],
        [ -3.50933229e-01,   8.85876862e-09,  -6.57515140e-10, ...,
          -1.80365926e-06,  -8.14464807e-06,   8.79454531e-09]]),
 'Theta2': array([[-0.76100352, -1.21244498, -0.10187131, -2.36850085, -1.05778129,
         -2.20823629,  0.56383834,  1.21105294,  2.21030997,  0.44456156,
         -1.18244872,  1

In [27]:
weights.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Theta1', 'Theta2'])

In [19]:
theta1, theta2 = weights['Theta1'], weights['Theta2']
print('theta1 :', theta1.shape)
print('theta2 :', theta2.shape)
params = np.r_[theta1.ravel(), theta2.ravel()]
print('params :', params.shape)

theta1 : (25, 401)
theta2 : (10, 26)
params : (10285,)


## Neural Networks - Feed Forward and Cost Computation

In [20]:
def sigmoid(z):
    return(1 / (1 + np.exp(-z)))

#### Sigmoid gradient
#### $$ g'(z) = g(z)(1 - g(z))$$
where $$ g(z) = \frac{1}{1+e^{-z}}$$

In [21]:
def sigmoidGradient(z):
    return(sigmoid(z)*(1-sigmoid(z)))

#### Cost Function 
#### $$ J(\theta) = \frac{1}{m}\sum_{i=1}^{m}\sum_{k=1}^{K}\big[-y^{(i)}_{k}\, log\,(( h_\theta\,(x^{(i)}))_k)-(1-y^{(i)}_k)\,log\,(1-h_\theta(x^{(i)}))_k)\big]$$

#### Regularized Cost Function
#### $$ J(\theta) = \frac{1}{m}\sum_{i=1}^{m}\sum_{k=1}^{K}\bigg[-y^{(i)}_{k}\, log\,(( h_\theta\,(x^{(i)}))_k)-(1-y^{(i)}_k)\,log\,(1-h_\theta(x^{(i)}))_k)\bigg] + \frac{\lambda}{2m}\bigg[\sum_{j=1}^{25}\sum_{k=1}^{400}(\Theta_{j,k}^{(1)})^2+\sum_{j=1}^{10}\sum_{k=1}^{25}(\Theta_{j,k}^{(2)})^2\bigg]$$

In [22]:
def nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, features, classes, reg):
    
    # When comparing to Octave code note that Python uses zero-indexed arrays.
    # But because Numpy indexing does not include the right side, the code is the same anyway.
    theta1 = nn_params[0:(hidden_layer_size*(input_layer_size+1))].reshape(hidden_layer_size,(input_layer_size+1))
    theta2 = nn_params[(hidden_layer_size*(input_layer_size+1)):].reshape(num_labels,(hidden_layer_size+1))

    m = features.shape[0]
    y_matrix = pd.get_dummies(classes.ravel()).as_matrix() 
    
    # Cost
    a1 = features # 5000x401
        
    z2 = theta1.dot(a1.T) # 25x401 * 401x5000 = 25x5000 
    a2 = np.c_[np.ones((features.shape[0],1)),sigmoid(z2.T)] # 5000x26 
    
    z3 = theta2.dot(a2.T) # 10x26 * 26x5000 = 10x5000 
    a3 = sigmoid(z3) # 10x5000
    
    J = -1*(1/m)*np.sum((np.log(a3.T)*(y_matrix)+np.log(1-a3).T*(1-y_matrix))) + \
        (reg/(2*m))*(np.sum(np.square(theta1[:,1:])) + np.sum(np.square(theta2[:,1:])))

    # Gradients
    d3 = a3.T - y_matrix # 5000x10
    d2 = theta2[:,1:].T.dot(d3.T)*sigmoidGradient(z2) # 25x10 *10x5000 * 25x5000 = 25x5000
    
    delta1 = d2.dot(a1) # 25x5000 * 5000x401 = 25x401
    delta2 = d3.T.dot(a2) # 10x5000 *5000x26 = 10x26
    
    theta1_ = np.c_[np.ones((theta1.shape[0],1)),theta1[:,1:]]
    theta2_ = np.c_[np.ones((theta2.shape[0],1)),theta2[:,1:]]
    
    theta1_grad = delta1/m + (theta1_*reg)/m
    theta2_grad = delta2/m + (theta2_*reg)/m
    
    return(J, theta1_grad, theta2_grad)

In [28]:
# Regularization parameter = 0
nnCostFunction(params, 400, 25, 10, X, y, 0)[0]

0.28762916516131892

In [44]:
# Regularization parameter = 1
nnCostFunction(params, 400, 25, 10, X, y, 1)[0]

0.38376985909092365

In [45]:
[sigmoidGradient(z) for z in [-1, -0.5, 0, 0.5, 1]]

[0.19661193324148185,
 0.23500371220159449,
 0.25,
 0.23500371220159449,
 0.19661193324148185]